In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
%matplotlib inline

In [4]:
df = pd.read_csv('./data/Customer-Churn.csv')

In [5]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [6]:
df.shape

(7043, 21)

In [15]:
def data_prep(df):
    
    df1 = df.copy()
    
    df1.columns = df1.columns.str.lower().str.replace(' ', '_')

    categorical_columns = list(df1.dtypes[df1.dtypes == 'object'].index)

    for col in categorical_columns:
        df1[col] = df1[col].str.lower().str.replace(' ', '_')
        
    y = np.where(df1.churn == 'yes', 1, 0)
    
    df1['totalcharges'] = pd.to_numeric(df1.totalcharges, errors='coerce')
    df1['totalcharges'] = df1['totalcharges'].fillna(0)
    df1['gender'] = np.where(df1['gender'] == 'female', 1, 0)
    df1['partner'] = np.where(df1['partner'] == 'yes', 1, 0)
    df1['dependents'] = np.where(df1['dependents'] == 'yes', 1, 0)
    df1['phoneservice'] = np.where(df1['phoneservice'] == 'yes', 1, 0)
    df1['multiplelines'] = np.where(df1['multiplelines'] == 'yes', 1, 0)
    df1['internetservicefiber'] = np.where(df1['internetservice'] == 'fiber_optic', 1, 0)
    df1['internetservicedsl'] = np.where(df1['internetservice'] == 'dsl', 1, 0)
    df1['onlinesecurity'] = np.where(df1['onlinesecurity'] == 'yes', 1, 0)
    df1['onlinebackup'] = np.where(df1['onlinebackup'] == 'yes', 1, 0)
    df1['deviceprotection'] = np.where(df1['deviceprotection'] == 'yes', 1, 0)
    df1['techsupport'] = np.where(df1['techsupport'] == 'yes', 1, 0)
    df1['streamingtv'] = np.where(df1['streamingtv'] == 'yes', 1, 0)
    df1['streamingmovies'] = np.where(df1['streamingmovies'] == 'yes', 1, 0)
    df1['paperlessbilling'] = np.where(df1['paperlessbilling'] == 'yes', 1, 0)
    df1 = pd.get_dummies(columns=['contract', 'paymentmethod'], data=df1, drop_first=True, dtype=int)

    columns_to_drop = ['churn', 'customerid', 'internetservice', 'gender', 'phoneservice', 'multiplelines']
    df1.drop(columns=columns_to_drop, inplace=True)
    X = df1.values
    return X, y

In [16]:
X, y = data_prep(df)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [21]:
df1.dtypes

seniorcitizen                                int64
partner                                      int64
dependents                                   int64
tenure                                       int64
onlinesecurity                               int64
onlinebackup                                 int64
deviceprotection                             int64
techsupport                                  int64
streamingtv                                  int64
streamingmovies                              int64
paperlessbilling                             int64
monthlycharges                             float64
totalcharges                               float64
internetservicefiber                         int64
internetservicedsl                           int64
contract_month-to-month                       bool
contract_one_year                             bool
contract_two_year                             bool
paymentmethod_bank_transfer_(automatic)       bool
paymentmethod_credit_card_(auto